<a href="https://colab.research.google.com/github/Bhanureddy48/Research-Methods-2/blob/main/ResearchMethods_Assignment2_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Data handling and visualization libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# String module for punctuation handling
import string
# Scikit-learn for splitting data into train and evaluation sets
from sklearn.model_selection import train_test_split
# Load T5 tokenizer and model for English-to-French translation
from transformers import T5Tokenizer, T5ForConditionalGeneration
# Hugging Face Datasets library for managing datasets
from datasets import Dataset
# Data collator for dynamic padding during training
from transformers import DataCollatorForSeq2Seq
# Install or upgrade Hugging Face Transformers library
!pip install -U transformers --quiet
!pip install --upgrade transformers --quiet
!pip install evaluate --quiet
# Install necessary metric dependencies for evaluate
!pip install sacrebleu rouge-score nltk --quiet
# Training configuration and training utilities from Transformers
from transformers import TrainingArguments, Trainer
# PyTorch for model input and device management
import torch
# tqdm for progress bars during generation
from tqdm import tqdm
# Hugging Face Evaluate library for BLEU, ROUGE, METEOR, chrF++ metrics
import evaluate

In [ ]:
# Define the URL to the English-French translation dataset
eng_to_fr_csv_url = "https://raw.githubusercontent.com/Bhanureddy48/Research-Methods-2/refs/heads/main/eng_-french.csv"
# Load only the first 20,000 rows from the dataset
eng_french_df = pd.read_csv(eng_to_fr_csv_url, nrows=20000)
# Rename columns for clarity
eng_french_df.columns = ["english_text", "french_text"]
# Display the first 5 English-French sentence pairs
print("Selected Set of English-French sentence pairs :")
print(eng_french_df.head())

In [ ]:
# Display basic information about the dataset
eng_french_df.info()

In [ ]:
# Show basic statistics for the English and French text columns
print(eng_french_df.describe(include="object"))

In [ ]:
# Check for missing (null) values in each column
print(eng_french_df.isnull().sum())

In [ ]:
# Check how many duplicate sentence pairs exist in the dataset
Eng_Fre_DupCnt = eng_french_df.duplicated().sum()
print(f"Number of duplicate rows: {Eng_Fre_DupCnt}")

In [ ]:
# Add sentence length columns (number of words) to the DataFrame
eng_french_df["english_length"] = eng_french_df["english_text"].apply(lambda x: len(x.split()))
eng_french_df["french_length"] = eng_french_df["french_text"].apply(lambda x: len(x.split()))
# Display basic statistics about sentence lengths
print("Sentence length statistics (word count):")
print(eng_french_df[["english_length", "french_length"]].describe())

In [ ]:
# Create a figure with 1 row and 2 columns
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# English sentence length distribution with density line
sns.histplot(eng_french_df["english_length"], bins=20, kde=True, ax=axes[0], color="skyblue", edgecolor="black")
axes[0].set_title("English Sentence Length (Word Count)")
axes[0].set_xlabel("Number of Words")
axes[0].set_ylabel("Frequency")
axes[0].grid(False)
# French sentence length distribution with density line
sns.histplot(eng_french_df["french_length"], bins=20, kde=True, ax=axes[1], color="lightgreen", edgecolor="black")
axes[1].set_title("French Sentence Length (Word Count)")
axes[1].set_xlabel("Number of Words")
axes[1].set_ylabel("Frequency")
axes[1].grid(False)
# Improve spacing between plots
plt.tight_layout()
plt.show()

In [ ]:
def clean_translation_pairs_dataframe(translation_df, min_char_count=2, max_word_limit=30, strip_punctuation=False):
    """
    This function cleans and preprocesses a DataFrame containing English-to-French
    sentence pairs. It standardizes the text by converting all characters to
    lowercase, trimming leading and trailing whitespace, and optionally removing
    punctuation. Sentence pairs that are too short or too long based on character
    count or word count thresholds are filtered out. The cleaned English and French
    sentences, along with their respective word counts, are stored in new columns.
    The final cleaned DataFrame is returned for use in modeling or further analysis.

    Parameters:
        translation_df (pd.DataFrame): The input DataFrame containing
            'english_text' and 'french_text' columns.
        min_char_count (int): Minimum number of characters required for both
            English and French sentences.
        max_word_limit (int): Maximum number of words allowed in each sentence.
        strip_punctuation (bool): If True, removes punctuation from both English
            and French text.

    Returns:
        pd.DataFrame: A cleaned DataFrame with the following columns:
            - 'english_text', 'french_text' (original sentences),
            - 'english_cleaned', 'french_cleaned' (preprocessed text),
            - 'english_sentence_length', 'french_sentence_length' (word counts).
    """
    # Work on a copy to avoid changing the original DataFrame
    cleaned_translation_df = translation_df.copy()
    # Convert sentences to lowercase and strip whitespace
    cleaned_translation_df["english_cleaned"] = cleaned_translation_df["english_text"].str.lower().str.strip()
    cleaned_translation_df["french_cleaned"] = cleaned_translation_df["french_text"].str.lower().str.strip()
    # remove punctuation from sentences
    if strip_punctuation:
        remove_punct_func = lambda txt: txt.translate(str.maketrans('', '', string.punctuation))
        cleaned_translation_df["english_cleaned"] = cleaned_translation_df["english_cleaned"].apply(remove_punct_func)
        cleaned_translation_df["french_cleaned"] = cleaned_translation_df["french_cleaned"].apply(remove_punct_func)
    # Filter out sentences that are shorter than the minimum character requirement
    cleaned_translation_df = cleaned_translation_df[
        (cleaned_translation_df["english_cleaned"].str.len() >= min_char_count) &
        (cleaned_translation_df["french_cleaned"].str.len() >= min_char_count)].reset_index(drop=True)
    # Calculate word counts for both English and French sentences
    cleaned_translation_df["english_sentence_length"] = cleaned_translation_df["english_cleaned"].apply(lambda x: len(x.split()))
    cleaned_translation_df["french_sentence_length"] = cleaned_translation_df["french_cleaned"].apply(lambda x: len(x.split()))
    # Filter out sentences that exceed the maximum word count
    cleaned_translation_df = cleaned_translation_df[
        (cleaned_translation_df["english_sentence_length"] <= max_word_limit) &
        (cleaned_translation_df["french_sentence_length"] <= max_word_limit)
    ].reset_index(drop=True)
    return cleaned_translation_df

In [ ]:
# Apply the translation cleaning function to the existing English-French DataFrame
cleaned_translation_df = clean_translation_pairs_dataframe(eng_french_df, strip_punctuation=True)
# Display original and cleaned sentence pairs
print(cleaned_translation_df[["english_text", "english_cleaned", "french_text", "french_cleaned"]].head())

In [ ]:
# Split cleaned English-French sentence pairs into training and evaluation sets (90% train, 10% eval)
train_translation_df, eval_translation_df = train_test_split(
    cleaned_translation_df[["english_cleaned", "french_cleaned"]],
    test_size=0.2,random_state=99999)
# Reset index for both training and evaluation DataFrames
train_translation_df = train_translation_df.reset_index(drop=True)
eval_translation_df = eval_translation_df.reset_index(drop=True)

In [ ]:
# Load the pretrained T5 tokenizer and model for English-to-French translation
translation_tokenizer = T5Tokenizer.from_pretrained("t5-small")
translation_model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:
# Convert training and evaluation DataFrames to Hugging Face Datasets
train_hf_dataset = Dataset.from_pandas(train_translation_df)
eval_hf_dataset = Dataset.from_pandas(eval_translation_df)
def tokenize_translation_pair(example):
    """
    Tokenizes a single English-to-French sentence pair for input into a T5 model.
    It prepends the translation prefix required by the model to the English input,
    tokenizes both English and French sentences, and aligns them as input and labels.

    Parameters:
        example (dict): A dictionary containing 'english_cleaned' and 'french_cleaned' keys.

    Returns:
        dict: A dictionary with tokenized input IDs, attention masks, and labels for training.
    """
    # Add the translation prefix expected by the T5 model
    source_text = "translate English to French: " + example["english_cleaned"]
    target_text = example["french_cleaned"]
    # Tokenize the source (English) sentence
    model_inputs = translation_tokenizer(
        source_text,max_length=128,
        padding="max_length",truncation=True)
    # Tokenize the target (French) sentence as the label
    with translation_tokenizer.as_target_tokenizer():
        label_tokens = translation_tokenizer(
            target_text,max_length=128,
            padding="max_length",truncation=True
        )
    # Assign tokenized labels to the model inputs
    model_inputs["labels"] = label_tokens["input_ids"]
    return model_inputs

In [ ]:
# Tokenize the training and evaluation datasets using the custom tokenization function
train_tokenized_dataset = train_hf_dataset.map(tokenize_translation_pair, batched=False)
eval_tokenized_dataset = eval_hf_dataset.map(tokenize_translation_pair, batched=False)

In [ ]:
# Data collator to dynamically pad inputs and labels for the translation task
translation_data_collator = DataCollatorForSeq2Seq(
    tokenizer=translation_tokenizer,model=translation_model)

In [ ]:
# Define training configuration for English-to-French translation using T5
translation_training_args = TrainingArguments(
    output_dir="./t5_translation_eng_to_fr_results",
    do_train=True,do_eval=True,per_device_train_batch_size=8,
    per_device_eval_batch_size=8,learning_rate=3e-4,
    num_train_epochs=3,weight_decay=0.01,
    logging_dir="./t5_translation_logs", logging_steps=50,
    save_total_limit=2, report_to="none")

In [ ]:
# Initialize the Trainer for English-to-French translation fine-tuning
translation_trainer = Trainer(
    model=translation_model,
    args=translation_training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=eval_tokenized_dataset,
    tokenizer=translation_tokenizer,         # Tokenizer used during training
    data_collator=translation_data_collator  # Handles dynamic padding
)

In [ ]:
# Start training the translation model
translation_train_output = translation_trainer.train()

In [ ]:
# Randomly select 200 samples from the evaluation set
translation_eval_df = eval_translation_df.sample(n=500, random_state=9999).reset_index(drop=True)
# Create task-formatted input strings for translation
eng_french_input_texts = [ "translate English to French: " + text
    for text in translation_eval_df["english_cleaned"].tolist()]
# Extract the ground truth French references
french_reference_texts = translation_eval_df["french_cleaned"].tolist()
# Tokenize the input prompts for model processing
eng_french_tokenized_inputs = translation_tokenizer(
    eng_french_input_texts,return_tensors="pt",
    padding=True,truncation=True,
    max_length=128).to(translation_model.device)
# Generate predictions from the translation model
with torch.no_grad():
    eng_french_generated_ids = translation_model.generate(
        input_ids=eng_french_tokenized_inputs["input_ids"],
        attention_mask=eng_french_tokenized_inputs["attention_mask"],
        max_length=128,num_beams=4)
# Decode generated token IDs into actual French sentences
french_predicted_texts = translation_tokenizer.batch_decode(eng_french_generated_ids,
    skip_special_tokens=True)
# Load evaluation metrics
eng_french_bleu_metric = evaluate.load("sacrebleu")
eng_french_rouge_metric = evaluate.load("rouge")
eng_french_chrf_metric = evaluate.load("chrf")
eng_french_meteor_metric = evaluate.load("meteor")
# Compute scores by comparing predicted and reference translations
eng_french_bleu_score = eng_french_bleu_metric.compute(predictions=french_predicted_texts,
    references=[[ref] for ref in french_reference_texts])["score"]
eng_french_rouge_score = eng_french_rouge_metric.compute(predictions=french_predicted_texts,
    references=french_reference_texts)["rougeL"]
eng_french_chrf_score = eng_french_chrf_metric.compute(predictions=french_predicted_texts,
    references=french_reference_texts)["score"]
eng_french_meteor_score = eng_french_meteor_metric.compute(predictions=french_predicted_texts,
                                  references=french_reference_texts)["meteor"]
# Pack evaluation results into a dictionary
eng_french_translation_scores = {
    "BLEU": eng_french_bleu_score,
    "ROUGE-L": eng_french_rouge_score,
    "chrF++": eng_french_chrf_score,
    "METEOR": eng_french_meteor_score
}
# Print formatted evaluation results
print("\nEvaluation on Sample Examples:")
for metric_name, score in eng_french_translation_scores.items():
    print(f"{metric_name}: {score:.2f}")

In [ ]:
# Create a DataFrame to compare input, reference, and predicted translations
sample_translation_output_df = pd.DataFrame({
    "English_Input": eng_french_input_texts[:50],
    "French_Reference": french_reference_texts[:50],
    "French_Predicted": french_predicted_texts[:50]
})
# Ensure full text is visible in each cell
pd.set_option('display.max_colwidth', None)
# Display the top 20 comparison results
sample_translation_output_df.head(50)